In [2]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# U-Net 모델 생성
def create_unet_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Contracting Path
    c1 = layers.Conv2D(64, (3, 3), activation=mish, padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation=mish, padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation=mish, padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation=mish, padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation=mish, padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation=mish, padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    b = layers.Conv2D(512, (3, 3), activation=mish, padding='same')(p3)
    b = layers.Conv2D(512, (3, 3), activation=mish, padding='same')(b)

    # Expanding Path
    u3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(b)
    u3 = layers.concatenate([u3, c3])
    c4 = layers.Conv2D(256, (3, 3), activation=mish, padding='same')(u3)
    c4 = layers.Conv2D(256, (3, 3), activation=mish, padding='same')(c4)

    u2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c4)
    u2 = layers.concatenate([u2, c2])
    c5 = layers.Conv2D(128, (3, 3), activation=mish, padding='same')(u2)
    c5 = layers.Conv2D(128, (3, 3), activation=mish, padding='same')(c5)

    u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c5)
    u1 = layers.concatenate([u1, c1])
    c6 = layers.Conv2D(64, (3, 3), activation=mish, padding='same')(u1)
    c6 = layers.Conv2D(64, (3, 3), activation=mish, padding='same')(c6)

    outputs = layers.GlobalAveragePooling2D()(c6)
    return Model(inputs, outputs)

# def create_mlp_model(input_shape):
#     input_layer = Input(shape=input_shape)
#     x = layers.Dense(128, activation=mish)(input_layer)
#     x = layers.Dense(64, activation=mish)(x)
#     x = layers.Dense(3, activation=mish)(x)
#     x = layers.Flatten()(x)
#     return Model(inputs=input_layer, outputs=x)

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# 데이터 로드 및 전처리
def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

# 모델 훈련 및 평가
def train_and_evaluate(folder_path, csv_path, valid_labels, test_subjects):
    accuracies = []
    for test_subject in test_subjects:
        print(f"Testing on subject: {test_subject}")
        train_df, test_df = prepare_data(folder_path, csv_path, test_subject, valid_labels)
        
        train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
        train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
        train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
        train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
        
        test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
        test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
        test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
        test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
        
        right_eye_model = create_unet_model((128, 128, 3))
        left_eye_model = create_unet_model((128, 128, 3))
        combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output])
        x = layers.Dense(256, activation=mish)(combined_input)
        x = layers.Dropout(0.5)(x)

        output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
        combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input], outputs=output_layer)

        combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        combined_model.fit(
            [train_images_array, train_images_array], train_labels,
            validation_data=([test_images_array, test_images_array], test_labels),
            epochs=50,
            batch_size=1,
            verbose=1
        )
        
        loss, accuracy = combined_model.evaluate([test_images_array, test_images_array], test_labels)
        accuracies.append(accuracy)
        print(f"Testing accuracy for subject {test_subject}: {accuracy:.4f}")
        
        # 결과 저장
        results = []
        for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
            test_df['Filename'], test_features, predicted_classes, actual_classes)):
            results.append({
                'Subject': test_subject,
                'Test Accuracy': accuracy,
                'Test Loss': loss,
                'Image File': image_path,
                'Predicted Class': pred_class,
                'Actual Class': actual_class
            })
        
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(csv_path, f"1127_UNet%notLandmark_results_{test_subject}.csv"), index=False, encoding='utf-8')
        print(f"Results for subject {test_subject} saved.")
        
        # 모델 저장
        model_save_path = os.path.join(csv_path, f"1127_UNet%notLandmark_model_{test_subject}.h5")
        combined_model.save(model_save_path)
        print(f"Model saved at: {model_save_path}")
    
    print(f"Average Accuracy: {np.mean(accuracies):.4f}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
test_subjects = ['lgj', 'hsb', 'scy']  # 테스트로 사용할 대상

valid_labels = [1, 5, 9, 19, 23, 27, 37, 41, 45]

# LOSO 수행
train_and_evaluate(folder_path, csv_path, valid_labels, test_subjects)


Testing on subject: lgj
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_25']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1800/1800 ━━━━━━━━━━━━━━━━━━━━ 933s 515ms/step - accuracy: 0.1189 - loss: 2.9531 - val_accuracy: 0.1111 - val_loss: 2.1989
Epoch 2/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 931s 517ms/step - accuracy: 0.1157 - loss: 8.4873 - val_accuracy: 0.1111 - val_loss: 2.1991
Epoch 3/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 930s 517ms/step - accuracy: 0.0961 - loss: 2.2046 - val_accuracy: 0.1111 - val_loss: 2.2012
Epoch 4/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 944s 524ms/step - accuracy: 0.0970 - loss: 2.2060 - val_accuracy: 0.1111 - val_loss: 2.1978
Epoch 5/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 953s 530ms/step - accuracy: 0.0943 - loss: 2.2052 - val_accuracy: 0.1111 - val_loss: 2.2022
Epoch 6/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 949s 527ms/step - accuracy: 0.1250 - loss: 2.2035 - val_accuracy: 0.1111 - val_loss: 2.1993
Epoch 7/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 940s 523ms/step - accuracy: 0.0906 - loss: 2.2165 - val_accuracy: 0.1111 - val_loss: 2.1990
Epoch 8/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 930s 516ms/step - accuracy: 0.1

NameError: name 'predicted_classes' is not defined